In [1]:
import json
import spacy
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, Vocab
from typing import List
from torch import nn
import torch
from einops import rearrange
import random
from tqdm import tqdm

In [2]:
with open('docs.json') as f:
    contents = f.read()
    docs = json.loads(contents)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gram = 7
bz = 256
epochs = 100
emb_dim = 64
dims = [256,]

In [4]:
def yield_tokens(docs):
    nlp = spacy.load("en_core_web_sm")
    for doc in docs:
        if "abstract" in doc:
            for abstract in doc["abstract"]:
                if "text" in abstract:
                    doc = nlp(abstract["text"])
                    assert doc.has_annotation("SENT_START")
                    for sent in doc.sents:
                        yield [token.lemma_.lower() for token in sent]

vocab = build_vocab_from_iterator(yield_tokens(docs), specials=["<unk>"], min_freq=10)
vocab.set_default_index(vocab["<unk>"])

vocab.__len__()

562

In [5]:
for ws in yield_tokens(docs):
    print(ws)
    break

['covid-19', ',', 'cause', 'by', 'sars', '-', 'cov-2', 'infection', ',', 'be', 'mild', 'to', 'moderate', 'in', 'the', 'majority', 'of', 'previously', 'healthy', 'individual', ',', 'but', 'can', 'cause', 'life', '-', 'threaten', 'disease', 'or', 'persistent', 'debilitate', 'symptom', 'in', 'some', 'case', '.']


In [6]:
datas = []
for ws in yield_tokens(docs):
    for i in range(n_gram//2, len(ws) - n_gram//2):
        ctx = [*ws[i-n_gram//2:i], *ws[i+1:i+1+n_gram//2],]
        tar = ws[i]
        datas.append((ctx, tar))

In [7]:
class CBOW(torch.nn.Module):
    def __init__(self, n_gram, emb_dim, dims, vocab,):
        super(CBOW, self).__init__()
        self.vocab = vocab
        self.n_gram = n_gram
        
        self.emb_book = nn.Embedding(vocab.__len__(), emb_dim)
        
        # dims = [emb_dim*(n_gram-1), *dims]
        # dims = [emb_dim*2, *dims]
        dims = [emb_dim, *dims]
        net = []
        for i in range(len(dims)-1):
            net.append(nn.Linear(dims[i], dims[i+1]))
            net.append(nn.ReLU(),)
            
        net.append(nn.Linear(dims[-1], vocab.__len__()),)
        net.append(nn.LogSoftmax(dim = -1),)
            
        self.net = nn.ModuleList(net)

    def forward(self, ctx_idx):
        inp = self.emb_book(ctx_idx)
        out = inp.mean(1)
        # out = rearrange(inp, "b n d -> b (n d)")
        # f, b = torch.chunk(inp, 2, 1)
        # out = torch.cat([f.mean(1), b.mean(1)], 1)
        for layer in self.net:
            out = layer(out)
        
        return out

In [8]:
model = CBOW(n_gram, emb_dim, dims, vocab).to(device)
loss_fn = nn.NLLLoss()

optim = torch.optim.Adamax([*model.parameters(),], lr=0.01)

In [9]:
for epoch in range(epochs):
    total_loss = 0
    random.shuffle(datas)
    bnum = len(datas)//bz + (1 if len(datas)%bz!=0 else 0)
    for bidx in tqdm(range(bnum)):
        ctx = []
        tar = []
        for _ctx, _tar in datas[bidx*bz:(bidx+1)*bz]:
            ctx.append(vocab(_ctx))
            tar.append(_tar)
        
        ctx = torch.tensor(ctx, dtype=torch.int64, device=device)
        tar = torch.tensor(vocab(tar), dtype=torch.int64, device=device)
        
        out = model(ctx)
        loss = loss_fn(out, tar)
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        total_loss += loss.item()*len(tar)
        
    print("epoch: {}, loss: {}".format(epoch, total_loss/len(datas)))
    torch.save({
        'epoch': epoch,
        'net_state_dict': model.state_dict(),
        'opt_state_dict': optim.state_dict(),
        }, "./cbow.pt")

 36%|███▌      | 41/114 [00:00<00:00, 408.84it/s]

epoch: 0, loss: 4.544087686358349


 35%|███▌      | 40/114 [00:00<00:00, 390.12it/s]

epoch: 1, loss: 4.041873703165209


 35%|███▌      | 40/114 [00:00<00:00, 391.37it/s]

...

epoch: 97, loss: 0.3194265909784772


 33%|███▎      | 38/114 [00:00<00:00, 372.16it/s]

epoch: 98, loss: 0.31357605352965445


100%|██████████| 114/114 [00:00<00:00, 341.63it/s]

epoch: 99, loss: 0.31065826108701955


In [10]:
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

emb_book = model.emb_book.weight.detach().cpu().numpy()
for idx, w in enumerate(vocab.get_itos()):
    emb = emb_book[idx]
    out_v.write('\t'.join([str(x) for x in emb]) + "\n")
    out_m.write(w + "\n")
out_v.close()
out_m.close()